# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [3]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key
print(geoapify_key)

3ed213feab8442a2b44c6ffe709fdbf0


## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [4]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Puerto Ayora,EC,74.55,few clouds,-0.7393,-90.3518,No hotel found
1,3,Isiro,CD,65.73,broken clouds,2.7739,27.6160,No hotel found
2,4,Porto Velho,BR,71.65,light rain,-8.7619,-63.9039,No hotel found
3,5,Hilo,US,82.27,few clouds,19.7297,-155.0900,No hotel found
4,8,Labuhan,ID,84.70,overcast clouds,-6.8844,112.2051,No hotel found
5,9,Kruisfontein,ZA,65.88,clear sky,-34.0033,24.7314,No hotel found
6,10,Sao Filipe,CV,72.81,few clouds,14.8961,-24.4956,No hotel found
7,15,Salalah,OM,69.89,clear sky,17.0151,54.0924,No hotel found
8,17,Torres,BR,74.82,overcast clouds,-29.3353,-49.7269,No hotel found
9,18,Victoria,HK,65.25,overcast clouds,22.2855,114.1577,No hotel found


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [5]:
# Configure the map plot
map_plot = map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel_name", "Country","Description"]
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [6]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = pd.DataFrame(vacation_df.loc[(vacation_df["City"] == "Bredasdorp")])
vacation_end   = pd.DataFrame(vacation_df.loc[(vacation_df["City"] == "Bredasdorp")])
vacation_stop1 = pd.DataFrame(vacation_df.loc[(vacation_df["City"] == "Hermanus")])
vacation_stop2 = pd.DataFrame(vacation_df.loc[(vacation_df["City"] == "Saldanha")])
vacation_stop3 = pd.DataFrame(vacation_df.loc[(vacation_df["City"] == "Calvinia")])

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [7]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end])

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
56,106,Bredasdorp,ZA,64.26,clear sky,-34.5322,20.0403,No hotel found
109,216,Hermanus,ZA,63.81,clear sky,-34.4187,19.2345,NaN
106,209,Saldanha,ZA,62.65,clear sky,-33.0117,17.9442,NaN
324,658,Calvinia,ZA,67.55,clear sky,-31.4707,19.7760,NaN
56,106,Bredasdorp,ZA,64.26,clear sky,-34.5322,20.0403,No hotel found


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [8]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[['Lat', 'Lng']].copy()

# Display sample data
waypoints_df

,Lat,Lng
56,-34.5322,20.0403
109,-34.4187,19.2345
106,-33.0117,17.9442
324,-31.4707,19.7760
56,-34.5322,20.0403


## Use GeoViews to create map that shows the four cities in the itinerary

In [9]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 150,
    color = 'City',
    hover_cols = ['Hotel Name', 'Country', 'Weather Description']
)

In [10]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [11]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [12]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '3ed213feab8442a2b44c6ffe709fdbf0',
 'waypoints': '-34.5322,20.0403|-34.4187,19.2345|-33.0117,17.9442|-31.4707,19.776|-34.5322,20.0403'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [13]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params).json()


## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [16]:
# Fetch the route's legs coordinates from the JSON reponse
leg1 = route_response['features'][0]['geometry']['coordinates'][0]
leg2 = route_response['features'][0]['geometry']['coordinates'][1]
leg3 = route_response['features'][0]['geometry']['coordinates'][2]
leg4 = route_response['features'][0]['geometry']['coordinates'][3]

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [17]:
# Create and empty list to store the longitude of each step
lng = []

# Create and empty list to store the latitude of step
lat = []


# Loop through the legs coordinates to fetch the latitude and longitude for each step
for item in leg1:
    lng.append(item[0])
    lat.append(item[1])
for item in leg2:
    lng.append(item[0])
    lat.append(item[1])
for item in leg3:
    lng.append(item[0])
    lat.append(item[1])
for item in leg4:
    lng.append(item[0])
    lat.append(item[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [18]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df['longitude'] = lng
route_df['latitude'] = lat

# Display sample data
route_df.head()

,longitude,latitude
0,20.040159,-34.532389
1,20.039917,-34.532266
2,20.039738,-34.532158
3,20.039222,-34.531858
4,20.039153,-34.531820


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [19]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(
    route_df,
).opts(
    frame_width=700,
    frame_height=500,
    line_width=3,
    color='black'
)

In [20]:
# Display a composed plot by using the route_map and route_path objects
waypoints_map * route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)
   .Path.I   :Path   [longitude,latitude]